pybaselines  
https://pybaselines.readthedocs.io/en/latest/  
conda install conda-forge::pybaselines

In [ ]:
import pandas
import numpy
from matplotlib import pyplot
filename = "dataframe.csv"
data = pandas.read_csv(filename, header=0, index_col=0).T
data.index = pandas.read_csv(filename, header=None, index_col=0).iloc[0].values
xmax, xmin = 2000, 1000
data = data.iloc[:, (xmin <= data.columns) & (data.columns <= xmax)]
data.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()

In [ ]:
# 1点でベースライン補正 / baseline correction by one point
x = 2000  # 補正点
y = data.iloc[:, numpy.abs(numpy.asarray(data.columns) - x).argmin()]
buff = (data.T - y).T
buff.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()
#data = buff

In [ ]:
# 1領域でベースライン補正 / baseline correction by one region
xmax, xmin = 2000, 1900  # 補正領域
y = data.iloc[:, (xmin < data.columns) & (data.columns < xmax)].T.min()
buff = (data.T - y).T
buff.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()
#data = buff

In [ ]:
# 2点でベースライン補正 / baseline correction by two points
xmax, xmin = 2000, 1900  # 補正点
y1 = data.iloc[:, numpy.abs(numpy.asarray(data.columns) - xmax).argmin()]
y2 = data.iloc[:, numpy.abs(numpy.asarray(data.columns) - xmin).argmin()]
a = (y2 - y1) / (xmin - xmax)
b = y1 - a * xmax
base = numpy.outer(a, data.columns) + numpy.array([b]).T
buff = pandas.DataFrame(data - base, index=data.index, columns=data.columns)
buff.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()
#data = buff

In [ ]:
# 2領域でベースライン補正 / baseline correction by two regions
xmax, xmin = 1700, 1500  # 補正領域1
buff = data.iloc[:, (xmin < data.columns) & (data.columns < xmax)]
x1, y1 = buff.idxmin(axis=1), buff.min(axis=1)
xmax, xmin = 1100, 1000  # 補正領域2
buff = data.iloc[:, (xmin < data.columns) & (data.columns < xmax)]
x2, y2 = buff.idxmin(axis=1), buff.min(axis=1)
a = (y2 - y1) / (x2 - x1)
b = y1 - a * x1
base = numpy.outer(a, data.columns) + numpy.array([b]).T
buff = pandas.DataFrame(data - base, index=data.index, columns=data.columns)
buff.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()
#data = buff

In [ ]:
# rubberband baseline correction
from scipy.spatial import ConvexHull
buff = data
x = data.columns
for i in range(len(data)):
	y = data.iloc[i].values
	v = ConvexHull(list(zip(x, y))).vertices
	v = numpy.roll(v, -v.argmin())
	v = v[:v.argmax()]
	buff.iloc[i] = data.iloc[i] - numpy.interp(x, x[v], y[v])
buff.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()
#data = buff

In [ ]:
# SNV
buff = ((data.T - data.T.mean()) / data.T.std()).T
buff.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()
#data = buff

In [ ]:
# MSC
x = numpy.matrix(data)
xc = x - x.mean(axis=1)
r = x.mean(axis=0)
rc = r - r.mean()
b = xc * rc.T * (rc * rc.T).I
buff = xc / b + r.mean()
buff = pandas.DataFrame(buff, index=data.index, columns=data.columns)
buff.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()
#data = buff

In [ ]:
# pybaselines
import pybaselines
x = data.columns
y = data.iloc[0].values
fitter = pybaselines.Baseline(x_data=x)
base, params = fitter.modpoly(y, poly_order=3)
#base, params = fitter.asls(y, lam=1e7, p=0.02)
#base, params = fitter.mor(y, half_window=30)
#base, params = fitter.snip(y, max_half_window=40, decreasing=True, smooth_half_window=3)
pyplot.plot(x, y)
pyplot.plot(x, base)
pyplot.show()
pyplot.plot(x, y - base)
pyplot.show()
buff = data.copy()
for i in range(len(buff)):
	y = buff.iloc[i].values
	base, params = fitter.modpoly(y, poly_order=3)
	buff.iloc[i] = y - base
buff.T.plot()
pyplot.gca().invert_xaxis()
pyplot.gca().legend_ = None
pyplot.show()
#data = buff